In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES']=str(1)

import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import csv
from keras.models import Sequential, Model
from keras.layers import Conv2D, ConvLSTM2D, Dense, MaxPooling2D, Dropout, Flatten, Reshape, merge, Input
from keras.optimizers import Adam

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('image_dir', '/notebooks/udacity/car_data/data/IMG/', 'Simulator Image data')
flags.DEFINE_string('data_path', '/notebooks/udacity/car_data/data/driving_log.csv', 'Simulator CSV')
flags.DEFINE_float('learn_rate', 0.0001, 'Trainign learning rate')

print ('Init completed')

with open(FLAGS.data_path, 'r') as f:
    reader = csv.reader(f)
    csv = np.array([row for row in reader][1:])
# center image, left image, right image
# steering (-0.8, 0.8)
# throttle (0, 1)
# brake (0, 1)
# speed (0, 9.8)

# Process single image
def proc_img(img): # input is 160x320x3
    img = img[59:138:2, 0:-1:2, :] # select vertical region and take each second pixel to reduce image dimensions
    img = (img / 127.5) - 1.0 # normalize colors from 0-255 to -1.0 to 1.0
    return img # return 40x160x3 image

# Read image names and remove IMG/ prefix
image_names_left = np.array(csv[:,1])
image_names_right = np.array(csv[:,2])
image_names_full = np.concatenate((image_names_left, image_names_right))
# read steering data and apply adjustment for left / right images
y_data = np.array(csv[:,3], dtype=float)
y_data_left = y_data+0.08
y_data_right = y_data-0.08
y_data_full = np.concatenate((y_data_left, y_data_right))
print ('CSV loaded')


from get_images import get_images

datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/',
                   '/notebooks/udacity/new_training/map1_error_correction/',
                   '/notebooks/udacity/new_training/map2_error_correction/'
         ]

images=get_images(datadirs,0.08)
#images=images[images.inverse==1]
#images=images[images.side!=0]
image_names_full, y_data_full = images.img.values, images.real.values

# Random sort for data and split test and validation sets
def newRandomTestValidationSplit(X, y):
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.01, random_state=111)
    return X_tr, X_val, y_tr, y_val

# Batch generator for training data
def generate_image_batch_tr(names, y_data, batch_size = 32):
    total_items = len(names)
    curr_item = 0
    while (True):
        image_data = np.zeros((batch_size,40,160, 3),dtype=float)
        steering_data = np.zeros((batch_size),dtype=float)
        for j in range(batch_size):
            image_name = names[curr_item]
            image = mpimg.imread(image_name)
            image_data[j] = proc_img(image)
            steering_data[j] = y_data[curr_item]
            curr_item = (curr_item+1)%total_items
        yield image_data, steering_data

# Batch generator for validation data (in this implementation same as for training data)
generate_image_batch=generate_image_batch_tr


Using TensorFlow backend.


Init completed
CSV loaded


/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# ----------------------
# Model - ideas from VG type network
inp = Input(shape=(40,160,3))
# First convolution is for model to determine the 'best' colorspace weights
x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(inp)
# Reduce dimensions
x = MaxPooling2D((2,2))(x) #20x80

# First convolution layer
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2))(x1) #10x40
x1 = Dropout(0.5)(x1)
flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

# Second convolution layer
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x2 = MaxPooling2D((2,2))(x2) #5x20
x2 = Dropout(0.5)(x2)
flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

# Second convolution layer
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
x3 = MaxPooling2D((2,2))(x3) #2x10
x3 = Dropout(0.5)(x3)
flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

# Merge the flattened ouputs after each convolution layer
x4 = merge([flat1, flat2, flat3], mode='concat')
# Fully connected layers
x5 = Dense(512, activation='relu')(x4)
x6 = Dense(128, activation='relu')(x5)
x7 = Dense(16, activation='relu')(x6)
out = Dense(1, activation='linear')(x7)

model = Model(input=inp, output=out)
model.summary()



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 40, 160, 3)    0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 40, 160, 3)    12          input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 20, 80, 3)     0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 20, 80, 32)    896         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [ ]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_test_images.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=1, mode='min')

# Compile, train and save
#model.compile(optimizer=Adam(lr=FLAGS.learn_rate), loss='mse')


model.compile(loss='mse', optimizer=Adam(lr=FLAGS.learn_rate), metrics=['mean_squared_error'])


print ('Split data')
X_tr_names, X_val_names, y_tr, y_val = newRandomTestValidationSplit(image_names_full, y_data_full)

print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history = model.fit_generator(generate_image_batch_tr(X_tr_names, y_tr, 64),samples_per_epoch=len(X_tr_names),
                              nb_epoch=100,
                              validation_data=generate_image_batch(X_val_names, y_val, 32),
                              nb_val_samples=len(y_val),  callbacks=[checkpoint, early_stop])

Split data
Start training
Epoch 1/100
272832/272859 [============================>.] - ETA: 0s - loss: 0.0568 - mean_squared_error: 0.0568   

/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_mean_squared_error improved from inf to 0.04586, saving model to model_test_images.h5
272896/272859 [==============================] - 288s - loss: 0.0568 - mean_squared_error: 0.0568 - val_loss: 0.0459 - val_mean_squared_error: 0.0459
Epoch 2/100
 36672/272859 [===>..........................] - ETA: 242s - loss: 0.0487 - mean_squared_error: 0.0487

In [12]:
#from car_models import modified_vgg
from keras.layers import Lambda

# ----------------------
# Model - ideas from VG type network
inp = Input(shape=(160,320,3))

x=Lambda(lambda img:img[59:138:2, 0:-1:2, :]/127.5-1.0,output_shape=(40,160,3))(inp)

# First convolution is for model to determine the 'best' colorspace weights
x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(x)
# Reduce dimensions
x = MaxPooling2D((2,2))(x) #20x80

# First convolution layer
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2))(x1) #10x40
x1 = Dropout(0.5)(x1)
flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

# Second convolution layer
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x2 = MaxPooling2D((2,2))(x2) #5x20
x2 = Dropout(0.5)(x2)
flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

# Second convolution layer
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
x3 = MaxPooling2D((2,2))(x3) #2x10
x3 = Dropout(0.5)(x3)
flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

# Merge the flattened ouputs after each convolution layer
x4 = merge([flat1, flat2, flat3], mode='concat')
# Fully connected layers
x5 = Dense(512, activation='relu')(x4)
x6 = Dense(128, activation='relu')(x5)
x7 = Dense(16, activation='relu')(x6)
out = Dense(1, activation='linear')(x7)

model = Model(input=inp, output=out)
model.summary()



# Batch generator for training data
def generate_image_batch_tr(names, y_data, batch_size = 32):
    total_items = len(names)
    curr_item = 0
    while (True):
        image_data = np.zeros((batch_size,160,320, 3),dtype=float)
        steering_data = np.zeros((batch_size),dtype=float)
        for j in range(batch_size):
            image_name = names[curr_item]
            image = mpimg.imread(image_name)
            image_data[j] = image
            steering_data[j] = y_data[curr_item]
            curr_item = (curr_item+1)%total_items
        yield image_data, steering_data

# Batch generator for validation data (in this implementation same as for training data)
generate_image_batch=generate_image_batch_tr

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 160, 320, 3)   0                                            
____________________________________________________________________________________________________
lambda_6 (Lambda)                (None, 40, 160, 3)    0           input_8[0][0]                    
____________________________________________________________________________________________________
convolution2d_43 (Convolution2D) (None, 40, 160, 3)    12          lambda_6[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_27 (MaxPooling2D)   (None, 20, 80, 3)     0           convolution2d_43[0][0]           
___________________________________________________________________________________________

In [13]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_test_images_modvgg.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=1, mode='min')

# Compile, train and save
#model.compile(optimizer=Adam(lr=FLAGS.learn_rate), loss='mse')


model.compile(loss='mse', optimizer=Adam(lr=FLAGS.learn_rate), metrics=['mean_squared_error'])


print ('Split data')
X_tr_names, X_val_names, y_tr, y_val = newRandomTestValidationSplit(image_names_full, y_data_full)

print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history = model.fit_generator(generate_image_batch_tr(X_tr_names, y_tr, 64),samples_per_epoch=len(X_tr_names),
                              nb_epoch=100,
                              validation_data=generate_image_batch(X_val_names, y_val, 32),
                              nb_val_samples=len(y_val),  callbacks=[checkpoint, early_stop])

Split data
Start training
Epoch 1/100


InvalidArgumentError: Matrix size-incompatible: In[0]: [3,83200], In[1]: [20480,512]
	 [[Node: MatMul_24 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](concat_v2_6, dense_25_W/read)]]
	 [[Node: Mean_14/_177 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1456_Mean_14", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'MatMul_24', defined at:
  File "/root/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/envs/carnd

In [7]:
help(Lambda)

Help on class Lambda in module keras.layers.core:

class Lambda(keras.engine.topology.Layer)
 |  Used for evaluating an arbitrary expressions on an input.
 |  
 |  # Examples
 |  
 |  ```python
 |      # add a x -> x^2 layer
 |      model.add(Lambda(lambda x: x ** 2))
 |  ```
 |  ```python
 |      # add a layer that returns the concatenation
 |      # of the positive part of the input and
 |      # the opposite of the negative part
 |  
 |      def antirectifier(x):
 |          x -= K.mean(x, axis=1, keepdims=True)
 |          x = K.l2_normalize(x, axis=1)
 |          pos = K.relu(x)
 |          neg = K.relu(-x)
 |          return K.concatenate([pos, neg], axis=1)
 |  
 |      def antirectifier_output_shape(input_shape):
 |          shape = list(input_shape)
 |          assert len(shape) == 2  # only valid for 2D tensors
 |          shape[-1] *= 2
 |          return tuple(shape)
 |  
 |      model.add(Lambda(antirectifier,
 |                       output_shape=antirectifier_output_shap